# $\color{#FF3399}{\textbf{Mini-project: Guess a Number (CLI) 🔢}}$

### $\color{#FF66B2}{\textbf{Task description}}$

Simple game CLI - "Guess a number (1-100)". User inputs their guesses, and the program responds with "too much/too little/guessed correctly".

### $\color{#FF66B2}{\textbf{Basic functions}}$

In [ ]:
import random

LOW, HIGH = 1, 100

# Setting random seed
RNG = random.Random() 
random_simulation_tries, binary_search_simulation_tries = [], []


def make_stats() -> dict[int, int, int | None, int]:
    """
    Simple stats kept in a dictionary.
    
    Structure:
      games: int           (number of played rounds)
      wins: int            (number of wins)
      best: Optional[int]  (lowest number of tries)
      total_tries: int     (sum of tries in all WINNED rounds)
    """
    return {"games": 0, "wins": 0, "best": None, "total_tries": 0}


def avg_tries(stats: dict[int, int, int | None, int]) -> float | None:
    """Average of tries in all WINNED rounds (or None)."""
    return (stats["total_tries"] / stats["wins"]) if stats["wins"] > 0 else None


def update_win(stats: dict[int, int, int | None, int], tries: int) -> None:
    """Increment games and wins; add tries to total_tries and update best (min)."""
    stats["games"] += 1
    stats["wins"] += 1
    stats["total_tries"] += tries
    
    if stats["best"] == None or stats["best"] == "-" or tries < stats["best"]:
        stats["best"] = tries;   


def update_loss(stats: dict[int, int, int | None, int]) -> None:
    """Increment only games."""
    stats["games"] += 1


def format_stats(stats: dict[int, int, int | None, int]) -> str:
    """
    Returns a short message with stats, eg.:
      G: 3 | W: 2 | BEST: 3 | AVG: 4.00
    When there's no avg/best, it shows '-'.
    
    Args:
        stats: dictionary of game stats
    
    Returns:
        str: formatted string diplaying current stats
    """
    avg_score = avg_tries(stats)
    
    if stats["best"] == None or avg_score == None:
        stats["best"], avg_score = "-", "-"
        
    return f"G: {stats["games"]} | W: {stats["wins"]} | BEST: {stats["best"]} | AVG: {avg_score}"


def evaluate_guess(secret: int, guess: int) -> int:
    """
    Guess evaluation. Result can be either: -1 (too less), 0 (correct guess), 1 (too much)} 
    
    Args:
        secret: number to be guessed
        guess: number given by the user/player
    
    Returns:
        int: score of evaluated guess
    """
    if guess > secret:
        return 1
    elif guess == secret:
        return 0
    else:
        return -1

# Game round (simple validation of I/O)
def play_round(low=LOW, high=HIGH, print_fn=print, input_fn=input, secret=None) -> int | None:
    """
    Play ONE round of the guessing game.
    
    Args:
        low: lower limit of guessed number's range
        high: upper limit of guessed number's range
        input_fn: function for user input
        print_fn: function for displaying messages
        secret: number to be guessed
    
    Returns:
        int:  number of tries (game won)
        None: game was interrupted ('q')
    """
    # Generate the number to be guessed
    if secret is None:
        secret = RNG.randint(low, high)

    tries = 0
    while True:
        raw = input_fn(f"Enter a number {low}-{high} (or 'q' to quit): ").strip()

        if raw.lower() in {"q", "quit"}:
            print_fn("Round aborted.")
            return None

        try:
            g = int(raw)
        except ValueError:
            print_fn("That’s not an integer.")
            continue

        if not (low <= g <= high):
            print_fn(f"Out of range {low}..{high}.")
            continue

        tries += 1
        cmp = evaluate_guess(secret, g)

        if cmp == 0:
            print_fn(f"✔ Correct! You guessed it in {tries} tries.")
            return tries

        print_fn("Too low." if cmp < 0 else "Too high.")

### $\color{#FF66B2}{\textbf{Bonus task description}}$


Add a **simulation mode** that compares at least two number-guessing strategies within the range LOW..HIGH, without user interaction. Implement the following:

1. **Random strategy** – in each attempt, choose a number randomly from the current range of possible values.

2. **Narrowing strategy** – choose the midpoint of the current interval (a “binary search” heuristic: after each “too low/too high” feedback, adjust the interval accordingly).

The program should run a **simulation of N games** (e.g., `N = 5000`) for each strategy and then compare the results.
Do not use classes or external libraries; store data in simple structures (like `dict` or `list`).
For reproducibility, you may set a fixed random seed.

After the simulation finishes, the program should display a clear summary for each strategy, including at least:

* The average number of attempts needed to guess correctly (rounded to 2 decimal places).

* The minimum and maximum number of attempts observed in the simulation.

* A short comparative conclusion (1–2 sentences), explaining which strategy performed better and by how much (e.g., “The narrowing strategy requires about ~x fewer attempts on average than the random one for the range 1–100.”).

### $\color{#FF66B2}{\textbf{Bonus task}}$

In [ ]:
def random_simulation(low=LOW, high=HIGH, print_fn=print, N=5000, secret=None):
    """
    Perform a random guessing simulation.

    Args:
        low: lower limit of guessed number's range
        high: upper limit of guessed number's range
        print_fn: function used to print messages
        N: maximum number of guesses allowed
        secret: the number to be guessed. If None, it will be randomly generated within the given range

    Returns:
        int: number of tries it took to guess the number.
        str: message indicating failure if the number wasn’t guessed within N attempts.
    """
    tries = 1

    if secret is None:
        secret = RNG.randint(low, high)

    for _ in range(N):
        guess = RNG.randint(low, high)
        evaluation = evaluate_guess(secret, guess)

        if evaluation == 0:
            print_fn(f"✔ I guessed it in {tries} tries.")
            random_simulation_tries.append(tries)
            return tries

        tries += 1
            
    return "Unfortunately, I couldn’t guess the number within {N} tries :("


def binary_search_simulation(low=LOW, high=HIGH, print_fn=print, N=5000, secret=None):
    """
    Perform a narrowing (binary search–style) guessing simulation.

    Args:
        low: lower limit of guessed number's range
        high: upper limit of guessed number's range
        print_fn: function used to print messages
        N: maximum number of guesses allowed
        secret: the number to be guessed. If None, it will be randomly generated within the given range

    Returns:
        int: number of tries it took to guess the number.
        str: message indicating failure if the number wasn’t guessed within N attempts.
    """
    tries = 1
    
    if secret is None:
        secret = RNG.randint(low, high)
        
    for _ in range(N):
        guess = (low + high) // 2
        evaluation = evaluate_guess(secret, guess)
        
        if evaluation == 0:
            print_fn(f"✔ I guessed it in {tries} tries.")
            binary_search_simulation_tries.append(tries)
            return tries
        elif evaluation == 1:
            high = guess - 1
        else:
            low = guess + 1
            
        tries += 1
            
    return "Unfortunately, I couldn’t guess the number within {N} tries :("

### $\color{#FF66B2}{\textbf{Main function - start a game (mini-menu)}}$

In [ ]:
def run_game():
    stats = make_stats()
    print("Witaj w grze LITE — komendy: n | player_stats | simulation_stats | q | random_sym | binary_sym")
    while True:
        cmd = input(">>> ").strip().lower()
        if cmd in {"q", "quit"}:
            break
        if cmd in {"player_stats"}:
            print(format_stats(stats)) 
            continue
        if cmd in {"simulation_stats"}:
            avg_random = sum(random_simulation_tries) / len(random_simulation_tries)
            avg_binary = sum(binary_search_simulation_tries) / len(binary_search_simulation_tries)

            print(f"Average number of attempts in the random strategy: {avg_random:.2f}")
            print(f"Minimum attempts: {min(random_simulation_tries)}, maximum attempts: {max(random_simulation_tries)}")

            print(f"\nAverage number of attempts in the narrowing (binary search) strategy: {avg_binary:.2f}")
            print(f"Minimum attempts: {min(binary_search_simulation_tries)}, maximum attempts: {max(binary_search_simulation_tries)}")

            print(f"\nOn average, the narrowing strategy requires {avg_random - avg_binary:.2f} fewer attempts than the random one for the range 1–100.")
            continue
        if cmd in {"random_sym"}:
            tries = random_simulation()
            continue
        if cmd in {"binary_sym"}:
            tries = binary_search_simulation()
            continue
        if cmd in {"n", "new", ""}:
            tries = play_round()
            if tries is None: 
                update_loss(stats)
            else: 
                update_win(stats, tries)
            continue
        print("Use: n | player_stats | simulation_stats | q | random_sym | binary_sym")

### $\color{#FF66B2}{\textbf{Basic tests and error tests}}$

In [ ]:
s = make_stats()
update_loss(s); assert s["games"] == 1 and s["wins"] == 0
update_win(s, 5); update_win(s, 3)
assert s["games"] == 3 and s["wins"] == 2 and s["best"] == 3
assert round(avg_tries(s), 2) == 4.00
assert evaluate_guess(50, 10) == -1
assert evaluate_guess(50, 90) ==  1
assert evaluate_guess(50, 50) ==  0

# without I/O
def fake_inputs(seq):
    it = iter(seq)
    return lambda prompt="": next(it)
out = []
tries = play_round(secret=7,
                   input_fn=fake_inputs(["3","10","7"]),
                   print_fn=lambda s: out.append(s))
assert tries == 3 and any("Trafione" in s for s in out)

run_game()